In [3]:
from stable_baselines3 import A2C,PPO
from tetris_gym.envs.tetris_gym import TetrisGym
from tetris_gym.wrappers.observation import ExtendedObservationWrapper
from tetris_gym.utils.eval_utils import evaluate, create_videos
from agent import agent

In [109]:
env = TetrisGym(width=10, height=20)

In [101]:
env = ExtendedObservationWrapper(env)

In [139]:
env.observation_space.sample()

OrderedDict([('board',
              array([[1, 0, 1, 1, 1, 0, 0, 1, 1, 1],
                     [1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
                     [1, 1, 0, 0, 0, 0, 1, 0, 1, 1],
                     [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                     [1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
                     [1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
                     [1, 1, 1, 1, 1, 1, 0, 0, 1, 1],
                     [0, 0, 1, 1, 1, 1, 0, 1, 0, 1],
                     [0, 0, 1, 1, 0, 0, 1, 1, 1, 1],
                     [0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
                     [0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
                     [1, 0, 0, 1, 0, 0, 1, 1, 0, 0],
                     [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
                     [1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
                     [1, 1, 1, 0, 1, 1, 1, 0, 0, 0],
                     [1, 1, 1, 0, 1, 1, 0, 1, 1, 1],
                     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                     [1, 1, 1, 1, 1, 0, 0, 1, 1, 0],
                     [1

In [214]:
env.reset()
obs, reward, done, info = env.step((0,0))

In [239]:
obs, reward, done, info = env.step((0,0))

In [243]:
obs

{'board': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'piece': 6,
 'heights': array([9, 9, 8, 5, 0, 0, 0, 0, 0, 0], dtype=int64),
 'bumps': [0, -1, -3, -5, 0, 0, 0, 0, 0]}

In [256]:
env.get_state_properties(obs['board'])

[0, 11, 9, 31]

# Whopperwhopperjuniorwhopper

In [595]:
class CustomRewardWrapper(gym.Wrapper):

    def __init__(self, env):
        super().__init__(env)

    # Felülírjük a környezet beépített step függvényét
    def step(self, action):

        # Meghívjuk az eredeti környezet step függvényét
        obs, reward, done, info = self.env.step(action)

        # Új jutalmat számítunk, minél jobban eldől az inga vagy elmozog a
        # kocsi, annál jobban büntetjük
        line_reward = 0.76
        hole_reward = 0.36
        bumpiness_reward = 0.18
        height_reward = 0.51
        lines_cleared = self.env.get_state_properties(obs['board'])[0]
        holes = self.env.get_state_properties(obs['board'])[1]
        bumpiness = self.env.get_state_properties(obs['board'])[2]
        height = self.env.get_state_properties(obs['board'])[3]
        reward += (- (hole_reward* holes )
                  - ( bumpiness_reward*bumpiness )
                  - (height_reward*height)
                  + (line_reward*lines_cleared)+2
        )
        return obs, reward, done, info

In [588]:
env = TetrisGym(width=10, height=20)
env = ExtendedObservationWrapper(env)
env = CustomRewardWrapper(env)

# Policy modification

## code

In [6]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym



In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
print(device)

cpu


### simple


In [9]:
board_size = env.observation_space['board'].shape

In [10]:
piece_size = env.observation_space['piece']

In [11]:
piece_size

Discrete(7)

In [12]:
a2_size = env.action_space[1].n

In [13]:
a1_size = env.action_space[0].n

In [15]:
s_size = board_size[0]*board_size[1]

In [ ]:
board_size,piece_size,heights_size,bumps_size

In [614]:
env.action_space.sample()[1]

0

In [599]:
class GoofyAhhNet(nn.Module):
    def __init__(self, s_size, a1_size, a2_size, h_size):
        super(GoofyAhhNet, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)

        self.fch1 = nn.Linear(h_size,h_size)

        
        self.fc21 = nn.Linear(h_size, a1_size)
        self.fc22 = nn.Linear(h_size, a2_size)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fch1(x))
        out1 = self.fc21(x)
        out2 = self.fc22(x)
        return [F.softmax(out1, dim=1),F.softmax(out2, dim=1)]
    
    def act(self, state):
        state = torch.from_numpy(state['board'].flatten()).float().unsqueeze(0).to(device)
        probs1 = self.forward(state)[0].cpu()
        probs2 = self.forward(state)[1].cpu()
        m1 = Categorical(probs1)
        action1 = m1.sample()
        m2 = Categorical(probs2)
        action2 = m2.sample()
        return (action1.item(),action2.item()),(m1.log_prob(action1),m2.log_prob(action2))
    def predict(self,state):
        action =self.act(state)
        act1,_ = action[0]
        act2,_ = action[1]
        return act1,act2

In [17]:
env.reset()['board'].flatten()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [18]:
a2_size

4

In [619]:
policy_net = GoofyAhhNet(s_size, a1_size,a2_size,100).to(device)
state = env.reset()

In [620]:
r1,r2 = policy_net(torch.from_numpy(state['board'].flatten()).float().unsqueeze(0).to(device))
r1 = r1.max(1)[1].view(1, 1)
r2 = r2.max(1)[1].view(1, 1)

In [621]:
r1

tensor([[6]])

In [622]:
r2

tensor([[1]])

In [618]:
debug_policy = GoofyAhhNet(s_size, a1_size,a2_size, 10).to(device)
debug_policy(torch.from_numpy(env.reset()['board'].flatten()).float().unsqueeze(0).to(device)).max(1)[1].view(1, 1)

AttributeError: 'list' object has no attribute 'max'

In [41]:
debug_policy

GoofyAhhNet(
  (fc1): Linear(in_features=200, out_features=10, bias=True)
  (fc21): Linear(in_features=10, out_features=10, bias=True)
  (fc22): Linear(in_features=10, out_features=4, bias=True)
)

In [70]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs1 = []
        saved_log_probs2 = []
        rewards = []

        state = env.reset()
        # Line 4 of pseudocode
        for t in range(max_t):
            actions, log_probs = policy.act(state)
            action1,action2 = actions
            log_prob1,log_prob2 = log_probs
            saved_log_probs1.append(log_prob1)
            saved_log_probs2.append(log_prob2)
            state, reward, done, _ = env.step((action1,action2))
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        
        eps = np.finfo(np.float32).eps.item()
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        
        for log_prob1,log_prob2, disc_return in zip(saved_log_probs1,saved_log_probs2 , returns):
            policy_loss.append(-log_prob1 *-log_prob2* disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

In [ ]:
tetris_hyperparams = {
    "h_size": 30,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "state_space": 200,
    "action_space": [a1_size,a2_size],
}

### extended

In [380]:
class ExtendedGoofyAhhNet(nn.Module):
    def __init__(self, s_size, a1_size, a2_size,h_size):
        super(ExtendedGoofyAhhNet, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size*2)
        self.fch1 = nn.Linear(h_size*2,h_size*2)
        self.fch2 = nn.Linear(h_size*2,h_size*2)
        self.fch3 = nn.Linear(h_size*2,h_size)
        self.fc21 = nn.Linear(h_size, a1_size)
        self.fc22 = nn.Linear(h_size, a2_size)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fch1(x))
        x = F.relu(self.fch2(x))
        x = F.relu(self.fch3(x))
        out1 = self.fc21(x)
        out2 = self.fc22(x)
        return [F.softmax(out1, dim=1),F.softmax(out2, dim=1)]
    
    def act(self, state):
        board_state = torch.from_numpy(state['board'].flatten()).float().unsqueeze(0).to(device)
        piece_state = torch.from_numpy(np.array([[state['piece']]])).float().to(device)
        height_state = torch.from_numpy(state['heights']).float().unsqueeze(0).to(device)
        bumps_state = torch.from_numpy(np.array(state['bumps'])).float().unsqueeze(0).to(device)
        state = torch.from_numpy(np.concatenate([board_state,piece_state,height_state,bumps_state],axis=1)).to(device)
        probs1 = self.forward(state)[0].cpu()
        probs2 = self.forward(state)[1].cpu()
        m1 = Categorical(probs1)
        action1 = m1.sample()
        m2 = Categorical(probs2)
        action2 = m2.sample()
        return (action1.item(),action2.item()),(m1.log_prob(action1),m2.log_prob(action2))
    def predict(self,state):
        action =self.act(state)
        act1,_ = action[0]
        act2,_ = action[1]
        return act1,act2

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs1 = []
        saved_log_probs2 = []
        rewards = []

        state = env.reset()
        # Line 4 of pseudocode
        for t in range(max_t):
            actions, log_probs = policy.act(state)
            action1,action2 = actions
            log_prob1,log_prob2 = log_probs
            saved_log_probs1.append(log_prob1)
            saved_log_probs2.append(log_prob2)
            state, reward, done, _ = env.step((action1,action2))
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        
        eps = np.finfo(np.float32).eps.item()
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        
        for log_prob1,log_prob2, disc_return in zip(saved_log_probs1,saved_log_probs2 , returns):
            policy_loss.append(-log_prob1 *-log_prob2* disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

In [388]:
board_size =env.observation_space.sample()['board'].shape[0] * env.observation_space.sample()['board'].shape[1]
piece_size = 1
height_size = env.observation_space.sample()['heights'].shape[0]
bumps_size = env.observation_space.sample()['bumps'].shape[0]

In [389]:
height_size

10

In [393]:
tetris_hyperparams = {
    "h_size": 100,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1500,
    "gamma": 1.0,
    "lr": 1e-2,
    "state_space": board_size+piece_size+height_size+bumps_size,
    "action_space": [a1_size,a2_size],
}

### convolutional

In [572]:
class GoofyAhhConvolutionalNet(nn.Module):
    def __init__(self, s_size, a1_size, a2_size, h_size):
        super(GoofyAhhConvolutionalNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)

        self.conv2 = nn.Conv2d(1, 1, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.flat = nn.Flatten()

        self.fc3 = nn.Linear(50, 10)
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.5)
        self.fch = nn.Linear(10,h_size)
        self.fc21 = nn.Linear(h_size,a1_size)
        self.fc22 = nn.Linear(h_size,a2_size)

    def forward(self, x):
        # input 3x32x32, output 32x32x32
        x = self.act1(self.conv1(x))
        x = self.drop1(x)
        # input 32x32x32, output 32x32x32
        x = self.act2(self.conv2(x))
        # input 32x32x32, output 32x16x16
        x = self.pool2(x)
        # input 32x16x16, output 8192
        x = self.flat(x)
        # input 8192, output 512
        x = self.act3(self.fc3(x))
        x = self.drop3(x)
        # input 512, output 10
        x = F.relu(self.fch(x))
        out1 = self.fc21(x)
        out2 = self.fc22(x)
        
        return [F.softmax(out1, dim=1),F.softmax(out2, dim=1)]
    
    def act(self, state):
        state = torch.from_numpy(state['board']).float().unsqueeze(0).to(device)
        probs1 = self.forward(state)[0].cpu()
        probs2 = self.forward(state)[1].cpu()
        m1 = Categorical(probs1)
        action1 = m1.sample((1,))
        m2 = Categorical(probs2)
        action2 = m2.sample((1,))
        return (action1.item(),action2.item()),(m1.log_prob(action1),m2.log_prob(action2))
    def predict(self,state):
        action =self.act(state)
        act1,_ = action[0]
        act2,_ = action[1]
        return act1,act2

In [551]:
o = torch.from_numpy(env.observation_space.sample()['board'].flatten()).float().unsqueeze(0).to(device)

In [559]:
o = torch.from_numpy(env.observation_space.sample()['board']).float().unsqueeze(0).to(device)

## teach

### Simple

In [607]:
tetris_hyperparams = {
    "h_size": 256,
    "n_training_episodes": 1500,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-4,
    "state_space": 200,
    "action_space": [a1_size,a2_size],
}

In [610]:
policy_net = GoofyAhhNet(tetris_hyperparams["state_space"], tetris_hyperparams["action_space"][0],tetris_hyperparams["action_space"][1], tetris_hyperparams["h_size"]).to(device)
target_net = GoofyAhhNet(tetris_hyperparams["state_space"], tetris_hyperparams["action_space"][0],tetris_hyperparams["action_space"][1], tetris_hyperparams["h_size"]).to(device)

In [ ]:
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.AdamW(policy_net.parameters(), lr=tetris_hyperparams['lr'], amsgrad=True)
steps_done = 0


In [608]:
tetris_policy = GoofyAhhNet(tetris_hyperparams["state_space"], tetris_hyperparams["action_space"][0],tetris_hyperparams["action_space"][1], tetris_hyperparams["h_size"]).to(device)
tetris_optimizer = optim.Adam(tetris_policy.parameters(), lr=tetris_hyperparams["lr"])

In [609]:
scores = reinforce(tetris_policy,
                   tetris_optimizer,
                   tetris_hyperparams["n_training_episodes"], 
                   tetris_hyperparams["max_t"],
                   tetris_hyperparams["gamma"], 
                   100)

Episode 100	Average Score: -124.36
Episode 200	Average Score: -117.26
Episode 300	Average Score: -118.34
Episode 400	Average Score: -118.34
Episode 500	Average Score: -116.20
Episode 600	Average Score: -117.43
Episode 700	Average Score: -118.29
Episode 800	Average Score: -114.48
Episode 900	Average Score: -115.47
Episode 1000	Average Score: -118.56
Episode 1100	Average Score: -117.16
Episode 1200	Average Score: -116.31
Episode 1300	Average Score: -115.82
Episode 1400	Average Score: -114.84
Episode 1500	Average Score: -116.86


### Extended

In [398]:
tetris_policy = ExtendedGoofyAhhNet(tetris_hyperparams["state_space"], tetris_hyperparams["action_space"][0],tetris_hyperparams["action_space"][1], tetris_hyperparams["h_size"]).to(device)
tetris_optimizer = optim.Adam(tetris_policy.parameters(), lr=tetris_hyperparams["lr"])

In [399]:
scores = reinforce(tetris_policy,
                   tetris_optimizer,
                   tetris_hyperparams["n_training_episodes"], 
                   tetris_hyperparams["max_t"],
                   tetris_hyperparams["gamma"], 
                   100)

Episode 100	Average Score: 974.17
Episode 200	Average Score: 843.17
Episode 300	Average Score: 850.02
Episode 400	Average Score: 844.39
Episode 500	Average Score: 836.65
Episode 600	Average Score: 837.89
Episode 700	Average Score: 860.49
Episode 800	Average Score: 836.40
Episode 900	Average Score: 849.62
Episode 1000	Average Score: 844.54


### Convo

In [592]:
tetris_hyperparams = {
    "h_size": 30,
    "n_training_episodes": 1500,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-1,
    "state_space": 200,
    "action_space": [a1_size,a2_size],
}

In [590]:
tetris_policy = GoofyAhhConvolutionalNet(tetris_hyperparams["state_space"], tetris_hyperparams["action_space"][0],tetris_hyperparams["action_space"][1], tetris_hyperparams["h_size"]).to(device)
tetris_optimizer = optim.Adam(tetris_policy.parameters(), lr=tetris_hyperparams["lr"])

In [596]:
scores = reinforce(tetris_policy,
                   tetris_optimizer,
                   tetris_hyperparams["n_training_episodes"], 
                   tetris_hyperparams["max_t"],
                   tetris_hyperparams["gamma"], 
                   100)

Episode 100	Average Score: -109.59
Episode 200	Average Score: -109.97
Episode 300	Average Score: -111.03
Episode 400	Average Score: -114.07
Episode 500	Average Score: -110.27
Episode 600	Average Score: -107.24
Episode 700	Average Score: -105.89
Episode 800	Average Score: -105.30
Episode 900	Average Score: -107.65
Episode 1000	Average Score: -108.63
Episode 1100	Average Score: -106.32
Episode 1200	Average Score: -107.32
Episode 1300	Average Score: -109.20
Episode 1400	Average Score: -108.56
Episode 1500	Average Score: -106.09


## eval

In [73]:
from tetris_gym.utils.eval_utils import evaluate_agent

In [74]:
score = evaluate_agent(env,tetris_policy, 10)
print("Score: {}".format(score))

Score: -931.1


# Defa Solution

In [111]:
model = PPO('MultiInputPolicy',  env, verbose=1, seed=42)
model.learn(total_timesteps=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 18.6     |
|    ep_rew_mean     | 759      |
| time/              |          |
|    fps             | 493      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------


In [112]:
# Kiértékelés 10 véletlen környezetben
score = evaluate(env,model, 10)
print("Score: {}".format(score))


Score: 396.17999999999995


# Video maker

In [387]:
!pip install imageio-ffmpeg


  Using cached imageio_ffmpeg-0.4.8-py3-none-win_amd64.whl (22.6 MB)


In [1]:
from tetris_gym.utils.eval_utils import create_agent_videos

In [602]:
create_agent_videos(env, tetris_policy)

-129.435